# 🛠️ Utilisation avancée des outils avec les modèles GitHub (.NET)

## 📋 Objectifs d'apprentissage

Ce notebook illustre des modèles d'intégration d'outils de niveau entreprise en utilisant le Microsoft Agent Framework en .NET avec les modèles GitHub. Vous apprendrez à créer des agents sophistiqués dotés de plusieurs outils spécialisés, en tirant parti du typage fort de C# et des fonctionnalités d'entreprise de .NET.

**Compétences avancées en outils que vous maîtriserez :**
- 🔧 **Architecture multi-outils** : Construire des agents avec plusieurs capacités spécialisées
- 🎯 **Exécution d'outils type-safe** : Exploiter la validation au moment de la compilation de C#
- 📊 **Modèles d'outils d'entreprise** : Conception d'outils prêts pour la production et gestion des erreurs
- 🔗 **Composition d'outils** : Combiner des outils pour des workflows métier complexes

## 🎯 Avantages de l'architecture d'outils .NET

### Fonctionnalités d'outils d'entreprise
- **Validation au moment de la compilation** : Le typage fort garantit la correction des paramètres des outils
- **Injection de dépendances** : Intégration du conteneur IoC pour la gestion des outils
- **Modèles Async/Await** : Exécution non bloquante des outils avec gestion appropriée des ressources
- **Journalisation structurée** : Intégration de journalisation intégrée pour le suivi de l'exécution des outils

### Modèles prêts pour la production
- **Gestion des exceptions** : Gestion complète des erreurs avec des exceptions typées
- **Gestion des ressources** : Modèles de disposition appropriés et gestion de la mémoire
- **Surveillance des performances** : Métriques intégrées et compteurs de performance
- **Gestion de la configuration** : Configuration type-safe avec validation

## 🔧 Architecture technique

### Composants principaux des outils .NET
- **Microsoft.Extensions.AI** : Couche d'abstraction unifiée pour les outils
- **Microsoft.Agents.AI** : Orchestration d'outils de niveau entreprise
- **Intégration des modèles GitHub** : Client API haute performance avec mise en pool des connexions

### Pipeline d'exécution des outils
```csharp
User Request → Agent Analysis → Tool Selection → Type Validation
                 ↓               ↓              ↓
         Parameter Binding → Tool Execution → Result Processing → Response
```

## 🛠️ Catégories et modèles d'outils

### 1. **Outils de traitement des données**
- **Validation des entrées** : Typage fort avec annotations de données
- **Opérations de transformation** : Conversion et formatage des données type-safe
- **Logique métier** : Outils de calcul et d'analyse spécifiques au domaine
- **Formatage des sorties** : Génération de réponses structurées

### 2. **Outils d'intégration**
- **Connecteurs API** : Intégration de services RESTful avec HttpClient
- **Outils de base de données** : Intégration d'Entity Framework pour l'accès aux données
- **Opérations sur les fichiers** : Opérations sécurisées sur le système de fichiers avec validation
- **Services externes** : Modèles d'intégration de services tiers

### 3. **Outils utilitaires**
- **Traitement de texte** : Manipulation et formatage de chaînes de caractères
- **Opérations sur les dates/heures** : Calculs de dates/heures respectueux des cultures
- **Outils mathématiques** : Calculs précis et opérations statistiques
- **Outils de validation** : Validation des règles métier et vérification des données

## ⚙️ Prérequis et configuration

**Environnement de développement :**
- SDK .NET 9.0 ou supérieur
- Visual Studio 2022 ou VS Code avec extension C#
- Accès à l'API des modèles GitHub

**Packages NuGet requis :**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**Configuration de l'environnement (fichier .env) :**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

Prêt à créer des agents de niveau entreprise avec des capacités d'outils puissants et type-safe en .NET ? Architecturons des solutions professionnelles ! 🏢⚡


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [ ]:
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [ ]:
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.3"

Installed Packages microsoft.agents.ai, 1.0.0-preview.251001.2

In [4]:
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [5]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
using System;
using System.ComponentModel;
using System.ClientModel;

using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;

In [7]:
 using DotNetEnv;

In [8]:
Env.Load("../../../.env");

In [9]:
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    var destinations = new List<string>
    {
        "Paris, France",
        "Tokyo, Japan",
        "New York City, USA",
        "Sydney, Australia",
        "Rome, Italy",
        "Barcelona, Spain",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bangkok, Thailand",
        "Vancouver, Canada"
    };

    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}

In [10]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [12]:

var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]);

In [14]:
AgentThread thread = agent.GetNewThread();

In [15]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip",thread));

Your day trip will be in Paris, France! Here's a proposed itinerary for your day:

### Morning
- **Breakfast at a Local Café**: Start your day with a traditional French breakfast. Try a croissant and café au lait at a local café.
- **Visit the Eiffel Tower**: Arrive early to beat the crowds. Enjoy the stunning views of the city from the top.

### Late Morning
- **Stroll Along the Seine**: Take a leisurely walk along the Seine River and enjoy the picturesque views.
- **Visit Notre-Dame Cathedral**: Explore this iconic Gothic cathedral and its stunning architecture.

### Lunch
- **Lunch at a Bistro**: Treat yourself to a classic French lunch at a nearby bistro. Consider trying coq au vin or a delicious quiche.

### Afternoon
- **Explore the Louvre**: Spend your afternoon at the Louvre Museum. While you may not see everything, be sure to check out the Mona Lisa and other famous artworks.
- **Walk Through the Tuileries Garden**: After the museum, relax in the beautiful Tuileries Garden.

#

In [16]:
Console.WriteLine(await agent.RunAsync("I don't like that destination. Plan me another vacation.",thread));

Your new vacation destination is Rio de Janeiro, Brazil! Here's a proposed itinerary for your trip:

### Day 1: Arrival in Rio
- **Check-in**: Arrive and settle into your accommodation.
- **Evening at Copacabana Beach**: Stroll along the famous Copacabana Beach. Enjoy the vibrant atmosphere and watch the sunset.

### Day 2: Exploring the City
- **Visit Christ the Redeemer**: Start your day with a visit to the iconic Christ the Redeemer statue. Take the train up to the top for breathtaking views.
- **Explore Santa Teresa**: Wander through the charming streets of Santa Teresa, known for its colonial-style houses, art studios, and local shops.
- **Lunch at a Local Restaurant**: Enjoy traditional Brazilian cuisine, such as feijoada (black bean stew).
- **Afternoon at Sugarloaf Mountain**: Take a cable car ride to Sugarloaf Mountain for panoramic views of the city and coastline.

### Day 3: Cultural Experience
- **Visit the Selarón Steps**: Walk up the colorful Selarón Steps, an iconic mosa


---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
